In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'SessionwiseGRU'
device = 'cuda:2'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class SessionwiseGRU(torch.nn.Module):
    def __init__(self, embedding, output_dim=1, dropout = 0.1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first = True,
            dropout=dropout
        )
        self.out_layer = torch.nn.Linear(embedding.embedding_dim, output_dim)


    def forward(self, batch):
        shp = batch['slates_item_indexes'].shape
        item_embs, user_embs = self.embedding(batch)
        item_embs = item_embs.flatten(-3, -2)
        
        # while training, let out model see the future 
        # while testing, it can see only the 
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)

#         print(indices.shape, user_embs.shape, item_embs.shape, )
        rnn_out, _ = self.rnn_layer(
            item_embs,
            user_embs,
        )
        return self.out_layer(rnn_out).reshape(shp)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SessionwiseGRU(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3669.56it/s]
/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.3333333134651184, 'accuracy': 0.25}


train... loss:0.7189432978630066:   0%|                                                                                                    | 1/5000 [00:00<27:56,  2.98it/s]

Val update: epoch: 0 |accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | treshold: 0.51
Test: accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | 


train... loss:0.6976741552352905:   0%|                                                                                                    | 3/5000 [00:01<30:40,  2.72it/s]

Val update: epoch: 3 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.54
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SessionwiseGRU(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (rnn_layer): GRU(32, 32, batch_first=True, dropout=0.1)
   (out_layer): Linear(in_features=32, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings
        ),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SessionwiseGRU with svd embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.17965421080589294, 'roc-auc': 0.5195829272270203, 'accuracy': 0.10167654603719711}


train... loss:47.48355868458748:   0%|                                                                                                  | 1/5000 [00:50<69:31:04, 50.06s/it]

Val update: epoch: 0 |accuracy: 0.09938888251781464 | f1: 0.18080750107765198 | auc: 0.5529186725616455 | treshold: 0.04
Test: accuracy: 0.09880203753709793 | f1: 0.17983591556549072 | auc: 0.5536670088768005 | 


...

train... loss:27.86328910291195:   0%|▍                                                                                                | 24/5000 [08:05<27:27:01, 19.86s/it]

Val update: epoch: 23 |accuracy: 0.8442241549491882 | f1: 0.3070088326931 | auc: 0.7279775142669678 | treshold: 0.18000000000000002
Test: accuracy: 0.8462962508201599 | f1: 0.3075462877750397 | auc: 0.7238301038742065 | 


train... loss:27.806676656007767:   0%|▍                                                                                               | 25/5000 [08:25<27:51:04, 20.15s/it]

Val update: epoch: 24 |accuracy: 0.8038306832313538 | f1: 0.3053048253059387 | auc: 0.7285906672477722 | treshold: 0.19
Test: accuracy: 0.8057520389556885 | f1: 0.30296483635902405 | auc: 0.7223894000053406 | 


train... loss:26.249242529273033:   2%|██▏                                                                                            | 113/5000 [37:00<26:40:32, 19.65s/it]


In [12]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['explicit', 'neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)


    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating SessionwiseGRU with explicit embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.4881993234157562, 'roc-auc': 0.42448124289512634, 'accuracy': 0.4352557063102722}


train... loss:60.50820517539978:   0%|                                                                                                  | 1/5000 [01:02<87:27:11, 62.98s/it]

Val update: epoch: 0 |accuracy: 0.7574372291564941 | f1: 0.8132645487785339 | auc: 0.8160781264305115 | treshold: 0.44
Test: accuracy: 0.7682505249977112 | f1: 0.8227155208587646 | auc: 0.8280416131019592 | 


...

train... loss:28.79174444079399:   2%|█▋                                                                                                | 87/5000 [09:50<9:17:36,  6.81s/it]

Val update: epoch: 86 |accuracy: 0.848135232925415 | f1: 0.8793451189994812 | auc: 0.9189528226852417 | treshold: 0.45
Test: accuracy: 0.8479264974594116 | f1: 0.8798685669898987 | auc: 0.9219239950180054 | 


train... loss:26.88394247740507:   3%|██▌                                                                                              | 129/5000 [14:24<9:01:24,  6.67s/it]

Val update: epoch: 128 |accuracy: 0.8463212847709656 | f1: 0.8835623860359192 | auc: 0.9192904233932495 | treshold: 0.44
Test: accuracy: 0.8479748368263245 | f1: 0.8852632641792297 | auc: 0.9212088584899902 | 


train... loss:26.445432737469673:   3%|██▉                                                                                             | 152/5000 [16:53<8:43:30,  6.48s/it]/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
train... loss:25.71717904508114:   4%|███▍                                                                                             | 179/5000 [19:54<8:56:24,  6.68s/it]


In [13]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items